<div style="text-align:center;"><img src="http://www.mf-data-science.fr/images/projects/intro.jpg" style='width:100%; margin-left: auto; margin-right: auto; display: block;' /></div>

# <span style="color: #641E16">Contexte</span>
Nous allons ici développer un algorithme de Machine Learning destiné à assigner automatiquement plusieurs tags pertinents à une question posée sur le célébre site Stack overflow.     
Ce programme s'adresse principalement aux nouveaux utilisateurs, afin de leur suggérer quelques tags relatifs à la question qu'ils souhaitent poser.

### Les données sources
Les données ont été cleanées dans le Notebook Kaggle [Stackoverflow questions - data cleaning](https://www.kaggle.com/michaelfumery/stackoverflow-questions-data-cleaning). Dans ce nettoyage ont par exemple été appliquées les techniques de stop words, suppression de la ponctuation et des liens, tokenisation, lemmatisation ...

### Objectif de ce Notebook
Dans ce Notebook, nous allons traiter la partie **modélisation des données textuelles avec des modèles supervisés et non supervisés**.     

Tous les Notebooks du projet seront **versionnés dans Kaggle mais également dans un repo GitHub** disponible à l'adresse https://github.com/MikaData57/Analyses-donnees-textuelles-Stackoverflow

# <span style="color:#641E16">Sommaire</span>
1. [Preprocessing : Bag of Words / Tf-Idf](#section_1)
2. [Modèles non supervisés](#section_2)     
    2.1. [Modèle LDA](#section_2_1)     
    2.2. [Modèle NMF](#section_2_2)     
3. [Modèles supervisés](#section_3)     
    3.1. [Régression logistique avec multi-labels](#section_3_1)      
    3.2. [Modélisation avec RandomForest](#section_3_2)       
    3.3. [Modèle RandomForest avec Classifier Chains](#section_3_3)       
    3.4. [Réseau de neurones avec Keras](#section_3_4)       

In [ ]:
# Install package for PEP8 verification
!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

In [ ]:
# Import Python libraries
import os
import warnings
import time
import numpy as np
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis

#RNN
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session
from keras import backend as K

# Library for PEP8 standard
from nbpep8.nbpep8 import pep8

In [ ]:
warnings.filterwarnings('ignore', category=DeprecationWarning)
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")
set_config(display='diagram')

In [ ]:
# Define path to data
path = '../input/stackoverflow-questions-filtered-2011-2021/'
data = pd.read_csv(path+"StackOverflow_questions_2009_2020_cleaned.csv",
                   sep=";", index_col=0,
                   converters={"Title": literal_eval,
                               "Body": literal_eval,
                               "Tags": literal_eval})
data.head(3)

In [ ]:
data.shape

Nous allons également créer une variable `Full_doc` qui accueillera le document complet de chaque item (Title et Body) :

In [ ]:
data["Full_doc"] = data["Title"] + data["Body"]
data["Full_doc"].head(3)

# <span style="color:#641E16" id="section_1">Preprocessing : Bag of Words / Tf-Idf</span>

Pour alimenter les modèles de machine learning, nous avons besoin de traiter des données numériques. Le modèle **Bag of Words** apprend un vocabulaire à partir de tous les documents, puis modélise chaque document en comptant le nombre de fois où chaque mot apparaît, convertissant donc les données textuelles en données numériques.

Nos données ayant déjà été cleanées et tokenisées dans le Notebook [stackoverflow-questions-data-cleaning](https://www.kaggle.com/michaelfumery/stackoverflow-questions-data-cleaning), nous allons initialiser l'algorithme du `CountVectorizer` sur les variables `Title` et `Body` *(X1 et X2)* sans preprocessing. Enfin, nous allons utiliser le module `TfidfVectorizer` de la librairie Scikit-Learn pour combiner le `CountVectorizer` et `TfidfTransformer`. Cela aura pour effet de pondérer la fréquence d'apparition des mots par un indicateur de similarité *(si ce mot est commun ou rare dans tous les documents)*. Dans cette partie, nous allons **éliminer les mots qui apparaissent dans plus de 60% des documents** (`max_df = 0.6`).

la métrique tf-idf ***(Term-Frequency - Inverse Document Frequency)*** utilise comme indicateur de similarité l'inverse document frequency qui est l'inverse de la proportion de document qui contient le terme, à l'échelle logarithmique.

Pour préparer nos targets *(pour les modèles supervisés)*, nous allons utiliser `MultiLabelBinarizer` de Scikit-Learn puisque nos `Tags` sont multiples.

In [ ]:
# Define X and y
X = data["Full_doc"]
y = data["Tags"]

# Initialize the "CountVectorizer" TFIDF for Full_doc
vectorizer = TfidfVectorizer(analyzer="word",
                             max_df=.6,
                             min_df=0.005,
                             tokenizer=None,
                             preprocessor=' '.join,
                             stop_words=None,
                             lowercase=False)

X_tfidf = vectorizer.fit_transform(X)

print("Shape of X for Full_doc: {}".format(X_tfidf.shape))

# Multilabel binarizer for targets
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(y)
y_binarized = multilabel_binarizer.transform(y)
print("Shape of y: {}".format(y_binarized.shape))

In [ ]:
# Create train and test split (30%)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_binarized,
                                                    test_size=0.3, random_state=8)
print("X_train shape : {}".format(X_train.shape))
print("X_test shape : {}".format(X_test.shape))
print("y_train shape : {}".format(y_train.shape))
print("y_test shape : {}".format(y_test.shape))

Comme les matrices sont relativment importantes, nous allons **vérifier le nombre de cellules qui ne sont pas à 0** :

In [ ]:
full_dense = X_tfidf.todense()
print("Full_doc sparsicity: {:.3f} %"\
      .format(((full_dense > 0).sum()/full_dense.size)*100))

Nous constatons que cette mesure est meilleure pour la varaible englobant Title et Body *(Full_doc)*.

# <span style="color:#641E16" id="section_2">Modèles non supervisés</span>

## <span id="section_2_1">Modèle LDA</span>
LDA, ou **Latent Derelicht Analysis** est un modèle probabiliste qui, pour obtenir des affectations de cluster, utilise deux valeurs de probabilité : $P(word | topics)$ et $P(topics | documents)$. Ces valeurs sont calculées sur la base d'une attribution aléatoire initiale, puis le calcul est répété pour chaque mot dans chaque document, pour décider de leur attribution de sujet. Dans cette méthode itérative, ces probabilités sont calculées plusieurs fois, jusqu'à la convergence de l'algorithme.

Nous allons entrainer 1 seul modèle basé sur la variable `Full_doc` en utilisant la librairie spécialisée **Gensim**. Pour cette partie, nous n'utiliserons pas le preprocessing TFIDF mais des fonctions propres aux méthodes Gensim.

Dans une première étape, le Bag of words est créé ainsi que la matrice de fréquence des termes dans les documents :

In [ ]:
# Create dictionnary (bag of words)
id2word = corpora.Dictionary(X)
id2word.filter_extremes(no_below=4, no_above=0.6, keep_n=None)
# Create Corpus 
texts = X  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

Gensim crée un identifiant unique pour chaque mot du document puis mappe word_id et word_frequency. Exemple : (6,3) ci-dessus indique que word_id 6 apparaît 3 fois dans le document et ainsi de suite.      
Les mots les plus fréquents ont ici aussi été filtrés grâce à la fonction `filter_extremes` réglée à 60% comme pour le Tfidf.

Pour voir quel mot correspond à un identifiant donné, il faut transmettre l'identifiant comme clé du dictionnaire. Exemple : id2word[4] :

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

Nous allons à présent entrainer le modèle LDA sur Full_doc puis afficher les métriques : 
- **Perplexity** : $(\exp(-1 \times \text{log-likelihood})$ *(Log likelihood : Densité de vraisemblance)*
- **Coherence Score** : Les mesures de cohérence de topics évaluent un seul topic en mesurant le degré de similitude sémantique entre les mots à score élevé dans ce dernier.       Pour en savoir plus sur ce Pipeline : [What is topic coherence ?](https://rare-technologies.com/what-is-topic-coherence/)

In [ ]:
# Build LDA model
full_lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=20,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
# Print Perplexity score
print('\nPerplexity: ', full_lda_model.log_perplexity(corpus))

#Print Coherence Score
coherence_model_lda = CoherenceModel(model=full_lda_model, 
                                     texts=texts, 
                                     dictionary=id2word, 
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### <span style="color:#641E16;">Visualisation des résultats de LDA Gensim sur Full_doc avec 20 topics</span>

In [ ]:
pyLDAvis.enable_notebook()
gensimvis.prepare(full_lda_model, corpus, id2word)

D'après les résulatas de cette modélisation LDA, il semble très **difficile de "nommer" les topics créés car les mots qui les composent sont très variés et sans fil conducteur clairement établi**. On voit cependant par exemple que le topic représenté par "server" englobe également "database", "sql", "connection" ou encore "query" ce qui est cohérent.

### Amélioration du modèle LDA

Cependant, dans l'algoritmes LDA, nous avons fixé arbitrairement à 20 le paramètre `num_topics` qui représente le nombre de topics à créer. Afin de sélectionner le meilleur nombre de topics pour nos données, nous allons **itérer sur une fourchette de nombre de topics et tester le score de cohérence pour chaque modèle** :

In [ ]:
# Iter LDA for best number of topics
coherence_test = []
for k in np.arange(1,90,10):
    print("Fitting LDA for K = {}".format(k))
    start_time = time.time()
    lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=k,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
    coherence_model_lda = CoherenceModel(model=lda_model,
                                         texts=texts,
                                         dictionary=id2word,
                                         coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    end_time = time.time()
    coherence_test.append((k, coherence_lda, (end_time - start_time)))

Affichons les scores des divers modèle :

In [ ]:
# Create dataframe of results
coherence_test = pd.DataFrame(coherence_test,
                              columns=["k","coherence","time"])

# Select best number of topics
best_nb_topics = coherence_test\
                    .loc[coherence_test.coherence.argmax(),"k"]

# Plot results
fig, ax1 = plt.subplots(figsize=(12,8))
x = coherence_test["k"]
y1 = coherence_test["coherence"]
y2 = coherence_test["time"]

ax1.plot(x, y1, label="Coherence score")
ax1.axvline(x=best_nb_topics, color='r', alpha=.7,
            linestyle='dashdot', label='Best param')
ax1.set_xlabel("Number of components")
ax1.set_ylabel("Coherence score")

ax2 = ax1.twinx()
ax2.plot(x, y2, label="Fit time",
         color='g', alpha=.5,
         linestyle='--')
ax2.set_ylabel("Fitting time (s)")

plt.title("Choosing Optimal LDA Model\n",
          color="#641E16", fontsize=18)
legend = fig.legend(loc=1, bbox_to_anchor=(.92, .9))

fig.tight_layout()
plt.show()

Testons à présent le modèle avec le meilleur nombre théorique de topics pour l'afficher avec LDAvis :

In [ ]:
# Best LDA visualization
# Construire le modèle LDA
best_lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=best_nb_topics,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
gensimvis.prepare(best_lda_model, corpus, id2word)

Pour attribuer des Tags à chaque question sur ces modèles non-supervisés, nous allons **créer une matrice Topic/Tags** en réalisant une multiplication matricielle des matrices Document / Topic et Document / Tags.

In [ ]:
# Calculate Document/topic matrix with Gensim
doc_topic = pd.DataFrame(best_lda_model.get_document_topics(corpus, minimum_probability=0))
for topic in doc_topic.columns:
    doc_topic[topic] = doc_topic[topic].apply(lambda x : x[1])

print('document/tag : ', y_binarized.shape)
print('document/topic : ', doc_topic.shape)

In [ ]:
# Print documents / topic matrix
doc_topic.head(3)

A présent, créons la matrice Topic / Tags grâce aux probabilités obtenues :

In [ ]:
# Matricial multiplication with Document / Topics transpose
topic_tag = np.matmul(doc_topic.T, y_binarized)
topic_tag.shape

In [ ]:
topic_tag

Nous obtenons donc une matrice dont les lignes représentent les Topics créés et les colonnes les Tags associés et leurs distribution. Nous allons donc **créer nos prédictions en prenant les** $\large n$ **premiers tags associés aux topics** de chaque document :

In [ ]:
y_results = pd.DataFrame(y)
y_results["best_topic"] = doc_topic.idxmax(axis=1).values
y_results["nb_tags"] = y_results["Tags"].apply(lambda x : len(x))

df_y_bin = pd.DataFrame(y_binarized)
df_dict = dict(
    list(
        df_y_bin.groupby(df_y_bin.index)
    )
)

tags_num = []
for k, v in df_dict.items():
    check = v.columns[(v == 1).any()]
    tags_num.append(check.to_list())

y_results["y_true"] = tags_num
y_results.head(3)

In [ ]:
# Select predicted tags in Topics / Tags matrix
list_tag = []
for row in y_results.itertuples():
    nb_tags = row.nb_tags
    best_topic = row.best_topic
    row_tags = list(topic_tag.iloc[best_topic].sort_values(ascending=False)[0:nb_tags].index)
    list_tag.append(row_tags)
    
y_results["y_pred"] = list_tag
y_results.head(3)

Nous allons tester plusieurs métriques sur ce modèle LDA :
- Accuracy score :
- F1 score :
- Jaccard similarity score : 
- Recall :
- Precision :

In [ ]:
def metrics_score(model, df, y_true, y_pred):
    if(df is not None):
        temp_df = df
    else:
        temp_df = pd.DataFrame(index=["Accuracy", "F1", "Jaccard", "Recall", "Precision"],
                               columns=[model])
        
    scores = []
    scores.append(metrics.accuracy_score(y_true, y_pred))
    scores.append(metrics.f1_score(y_pred, y_true, average='weighted'))
    scores.append(metrics.jaccard_score(y_true, y_pred, average='weighted'))
    scores.append(metrics.recall_score(y_true, y_pred, average='weighted'))
    scores.append(metrics.precision_score(y_true, y_pred, average='weighted'))
    temp_df[model] = scores
    
    return temp_df

metrics_score("LDA", df=None, 
              y_true=y_results.y_true, 
              y_pred=y_results.y_pred)

On remarque ici que la modélisation non supervisée avec LDA n'est pas adaptée. En effet, le meilleur nombre de topics se situerait à 31, mais **l'algorithme ne parvient pas a établir de groupes bien distincts**. Un certain nombre de topics sont très regroupés et donc représentés par les mêmes termes.

Nous allons donc tester une seconde modélisation non supervisée.

## <span id="section_2_2">Modèle NMF</span>
<details>
  <summary style="color:blue;">Explication du modèle</summary>
  
  ## NMF
  La factorisation matricielle non négative *(**N**on-negative **M**atrix **F**actorization)* est un modèle linéaire-algéabrique, qui factorise des vecteurs de grande dimension dans une représentation de faible dimension. Similaire à l'analyse en composantes principales *(PCA)*, NMF profite du fait que **les vecteurs sont non négatifs**. En les factorisant dans la forme de dimension inférieure, NMF force les coefficients à être également non négatifs.<br/><br/>
    Prenons une matrice d'origine $A$, nous pouvons obtenir deux matrices $W$ et $H$, telles que $A = WH$. NMF a une propriété de clustering, telle que $W$ et $H$ représentent les informations suivantes sur $A$ :
    <ul><li>$A$ (Matrice Document-word) : Matrice qui contient "quels mots apparaissent dans quels documents".</li>
    <li>$W$ (Vecteurs de base) : Topics découverts à partir des documents.</li>
    <li>$H$ (Matrice de coefficients) : les poids pour les topics dans chaque document.</li></ul><br/>
    Nous calculons $W$ et $H$ en optimisant sur une **fonction objectif**, en mettant à jour à la fois $W$ et $H$ de manière itérative jusqu'à convergence.</br></br>
    $$\large \frac{1}{2} ||A - WH||^2_F = \sum_{i=1}^{n} \sum_{j=1}^{m} (A_{ij} - (WH)_{ij})^2$$</br>
    Dans cette fonction objectif, nous mesurons l'erreur de reconstruction entre A et le produit de ses facteurs W et H, en fonction de la distance euclidienne. Les valeurs mises à jour sont calculées dans des opérations parallèles, et en utilisant les nouveaux W et H, nous recalculons l'erreur de reconstruction, en répétant ce processus jusqu'à la convergence.
</details>

Le modèle **NMF ne peut malheureusement pas être scoré**. Nous allons donc nous baser sur les résultats de la LDA pour déterminer un nombre correct de composants. Ici, nous prendrons **31 topics** pour avoir un bon compromis "temps d'entrainement" / précision et utiliserons les matrices Tfidf créées lors du preprocessing.

In [ ]:
def plot_top_words(model, feature_names, n_top_words, nb_topic_plot, title):
    """Function for displaying the plots of the 
    best x words representative of the categories of NMF.

    Parameters
    ----------------------------------------
    model : NMF model
        Fitted model of NMF to plot
    feature_names : array
        Categories result of the vectorizer (TFIDF ...)
    n_top_words : int
        Number of words for each topic.
    title : string
        Title of the plot.
    ----------------------------------------
    """
    rows = int(nb_topic_plot/6)
    fig, axes = plt.subplots(rows, 6, 
                             figsize=(30, rows*10), 
                             sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        if(topic_idx < nb_topic_plot):
            top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
            top_features = [feature_names[i] for i in top_features_ind]
            weights = topic[top_features_ind]

            ax = axes[topic_idx]
            bartopic = ax.barh(top_features, weights, height=0.7)
            bartopic[0].set_color('#f48023')
            ax.set_title(f'Topic {topic_idx +1}',
                         fontdict={'fontsize': 30})
            ax.invert_yaxis()
            ax.tick_params(axis='both', which='major', labelsize=20)
            for i in 'top right left'.split():
                ax.spines[i].set_visible(False)
            fig.suptitle(title, fontsize=36, color="#641E16")

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
# Define number of topics to test
n_topics = best_nb_topics

print("-"*50)
print("Start NMF fitting on Full_doc ...")
print("-" * 50)
start_time = time.time()
# Initializing the NMF
full_nmf = NMF(n_components=n_topics,
               init='nndsvd',
               random_state=8)

# Fit NMF on Body vectorized
full_nmf.fit(X_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

# Plot the 12 first topics
ff_feature_names = vectorizer.get_feature_names()
plot_top_words(full_nmf, ff_feature_names, 20, 12,
               'Topics in NMF model for Full_doc')

**La modélisation avec NMF nous apporte des catégories aussi lisibles que celles de l'algorithme LDA**. 1 mot est toujours beaucoup plus représentaif de cette catégorie mais les regroupements sont globalement cohérents. Un topic par exemple illustre bien les sujets liés SQL, aux requêtes, un second traite les sujets liés aux dictionnaires ...

En revanche, les topics générés restent très généraux et ne permettent pas une catégorisation cohérente pour notre problème d'auto-tagging. Nous allons donc tester des modèles supervisés.

# <span style="color:#641E16" id="section_3">Modèles supervisés</span>

## <span id="section_3_1">Régression logistique avec multi-labels</span>
Nous avons déjà réalisé quelques Notebook traitant de la régression logistique : c'est une technique prédictive. Elle vise à construire un modèle permettant de prédire / expliquer les valeurs prises par une variable cible qualitative à partir d’un ensemble de variables explicatives quantitatives ou qualitatives encodées.

Pour cette partie sur les modélisations supervisées, nous allons utiliser la variable `Full_doc` qui regroupe le Title et le Body puis créer un Pipeline qui ne pourra pas inclure la transformation de notre variable cible *(MultiLabelBinarizer ne fonctionne pas dans les Pipeline SKlearn)*.

In [ ]:
# Initialize Logistic Regression with OneVsRest
param_logit = {"estimator__C": [100, 10, 1.0, 0.1],
               "estimator__penalty": ["l1", "l2"],
               "estimator__dual": [False],
               "estimator__solver": ["liblinear"]}

multi_logit_cv = GridSearchCV(OneVsRestClassifier(LogisticRegression()),
                              param_grid=param_logit,
                              n_jobs=-1,
                              cv=5,
                              scoring="f1_weighted",
                              return_train_score = True,
                              refit=True)
multi_logit_cv.fit(X_train, y_train)

In [ ]:
logit_cv_results = pd.DataFrame.from_dict(multi_logit_cv.cv_results_)
print("-"*50)
print("Best params for Logistic Regression")
print("-" * 50)
logit_best_params = multi_logit_cv.best_params_
print(logit_best_params)

In [ ]:
logit_cv_results[logit_cv_results["params"]==logit_best_params]

Nous pouvons maintenant **réaliser les prédictions avec le modèle de régression logistique sur le jeu de test** pour pouvoir les comparer avec le jeu y_test.

In [ ]:
# Predict
y_test_predicted_labels_tfidf = multi_logit_cv.predict(X_test)

# Inverse transform
y_test_pred_inversed = multilabel_binarizer.inverse_transform(y_test_predicted_labels_tfidf)
y_test_inversed = multilabel_binarizer.inverse_transform(y_test)

print("-"*50)
print("Print 5 first predicted Tags vs true Tags")
print("-" * 50)
print("Predicted:", y_test_pred_inversed[0:5])
print("True:", y_test_inversed[0:5])

Puis nous **calculons les diverses métriques sur le meilleur modèle** de régression logistique :

In [ ]:
df_metrics_compare = metrics_score("Logit", df=None, 
              y_true = y_test, 
              y_pred = y_test_predicted_labels_tfidf)
df_metrics_compare

## <span id="section_3_2">Modélisation avec RandomForest</span>

In [ ]:
# Initialize RandomForest with OneVsRest
param_rfc = {"estimator__max_depth": [5, 25, 50],
             "estimator__min_samples_leaf": [1, 5, 10],
             "estimator__class_weight": ["balanced"]}

multi_rfc_cv = GridSearchCV(OneVsRestClassifier(RandomForestClassifier()),
                                  param_grid=param_rfc,
                                  n_jobs=-1,
                                  cv=2,
                                  scoring="f1_weighted",
                                  return_train_score = True,
                                  refit=True,
                                  verbose=3)
# Fit on Sample data
multi_rfc_cv.fit(X_train[0:7000], y_train[0:7000])

In [ ]:
rfc_cv_results = pd.DataFrame.from_dict(multi_rfc_cv.cv_results_)
print("-"*50)
print("Best params for RandomForestClassifier")
print("-" * 50)
rfc_best_params = multi_rfc_cv.best_params_
print(rfc_best_params)

In [ ]:
rfc_best_params_ok = {}
for k, v in rfc_best_params.items():
    rfc_best_params_ok[k.replace("estimator__","")] = v

In [ ]:
# Refit RandomForestClassifier best_params with full dataset
rfc_final_model = OneVsRestClassifier(RandomForestClassifier(**rfc_best_params_ok))
rfc_final_model.fit(X_train, y_train)

# Predict
y_test_predicted_labels_tfidf_rfc = rfc_final_model.predict(X_test)

# Inverse transform
y_test_pred_inversed_rfc = multilabel_binarizer.inverse_transform(y_test_predicted_labels_tfidf_rfc)

print("-"*50)
print("Print 5 first predicted Tags vs true Tags")
print("-" * 50)
print("Predicted:", y_test_pred_inversed_rfc[0:5])
print("True:", y_test_inversed[0:5])

In [ ]:
df_metrics_compare = metrics_score("RandomForest", df=df_metrics_compare, 
              y_true = y_test, 
              y_pred = y_test_predicted_labels_tfidf_rfc)
df_metrics_compare

Les métriques sur le modèle RandomForest sont moins bonnes mais semblent cependant plus cohérente avec les données, ce d'autant que les métriques Jaccard et F1 sont proches. D'autre part, nous pouvons **vérifier le nombre de lignes dont les Tags ne sont pas prédit** afin de voir si l'un des modèles est meilleur :

In [ ]:
Tags_per_row_lr = y_test_predicted_labels_tfidf.sum(axis=1)
null_rate_lr = round(((Tags_per_row_lr.size - np.count_nonzero(Tags_per_row_lr))
                      /Tags_per_row_lr.size)*100,2)
Tags_per_row_rfc = y_test_predicted_labels_tfidf_rfc.sum(axis=1)
null_rate_rfc = round(((Tags_per_row_rfc.size - np.count_nonzero(Tags_per_row_rfc))
                       /Tags_per_row_rfc.size)*100,2)
print("-"*50)
print("Percentage of non tagged question for each model")
print("-" * 50)
print("Logistic Regression: {}%".format(null_rate_lr))
print("Random Forest: {}%".format(null_rate_rfc))

Le Random Forest semble donc plus approprié à notre programme d'auto-tagging sur les données Stackoverflow. Nous allons à présent **tester ce modèle RandomForest avec Classifier Chains** pour remplacer la méthode One versus rest :

## <span id="section_3_3">Modèle RandomForest avec Classifier Chains</span>

Avec la méthode ClassifierChains, chaque modèle fait une prédiction dans l'ordre spécifié en utilisant toutes les fonctionnalités disponibles fournies au modèle mais également les prédictions des modèles précédents.

In [ ]:
rfc_base_model = RandomForestClassifier(**rfc_best_params_ok)
chain = ClassifierChain(rfc_base_model, order='random')
chain.fit(X_train, y_train)

In [ ]:
# Predict
y_test_predicted_labels_tfidf_chain = chain.predict(X_test)

# Inverse transform
y_test_pred_inversed_chain = multilabel_binarizer.inverse_transform(y_test_predicted_labels_tfidf_chain)

print("-"*50)
print("Print 5 first predicted Tags vs true Tags")
print("-" * 50)
print("Predicted:", y_test_pred_inversed_chain[0:5])
print("True:", y_test_inversed[0:5])

In [ ]:
Tags_per_row_chain = y_test_predicted_labels_tfidf_chain.sum(axis=1)
null_rate_chain = round(((Tags_per_row_chain.size - np.count_nonzero(Tags_per_row_chain))
                       /Tags_per_row_chain.size)*100,2)
print("-"*50)
print("Percentage of non tagged question for chain model")
print("-" * 50)
print("RandomForest with Classifier Chains: {}%".format(null_rate_chain))

In [ ]:
df_metrics_compare = metrics_score("RFC Chains", df=df_metrics_compare, 
              y_true = y_test, 
              y_pred = y_test_predicted_labels_tfidf_chain)
df_metrics_compare

Le modèle RandomForest avec Classifier Chains offre des métriques similaires au modèle avec OneVsRest mais le **taux de remplissage des valeurs prédites est encore meilleur**.

Nous allons à présent tester un dernier modèle d'apprentissage profond avec Keras et un réseau de neurones simple.

## <span id="section_3_4">Réseau de neurones avec Keras</span>

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def jaccard_m(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

Nous allons définir une fonction pour constuire le réseau de neurones assez simple. RNN avec une couche cachée et complétement connectée. Nous utiliserons également un Dropout pour éviter le sur-apprentissage.

In [ ]:
def build_rnn(input_dim, hidden_neurons, output_dim):
    """
    Construct a Keras model which will be used to 
    fit/predict in SKlearn pipeline.
    """
    # Create brain
    model = Sequential()
    model.add(layers.Dense(hidden_neurons, input_dim=input_dim, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(hidden_neurons, input_dim=input_dim, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(output_dim, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', recall_m, precision_m, f1_m, jaccard_m])
    model.summary()
    
    return model

In [ ]:
clear_session()

model_params = {
    'build_fn': build_rnn,
    'input_dim': X_train.shape[1],
    'hidden_neurons': 150,
    'output_dim': y_train.shape[1],
    'epochs': 20,
    'batch_size': 256,
    'verbose': 0,
    'validation_data': (X_test.toarray(), y_test),
    'shuffle': True}

rnn_model = KerasClassifier(**model_params)

In [ ]:
history = rnn_model.fit(X_train.toarray(), y_train)

In [ ]:
# evaluate the model
scores = rnn_model.score(X_test.toarray(), y_test)
scores

In [ ]:
train_accuracy = history.history.get('accuracy', [])
train_f1 = history.history.get('f1_m', [])
val_accuracy = history.history.get('val_accuracy', [])
val_f1 = history.history.get('val_f1_m', [])

fig, axes = plt.subplots(1, 2, figsize=(25, 8))
axes[0].plot(np.arange(0,20,1),
             train_accuracy,
             label="Train")
axes[0].plot(np.arange(0,20,1),
             val_accuracy,
             linestyle='--', color='g', alpha=.7,
             label="Validation")
axes[0].set_xticks(np.arange(0,20,5))
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Accuracy score")
axes[0].set_title('Model accuracy through epochs',
                  color='#f48023', fontweight='bold')
axes[0].legend(loc=4)

axes[1].plot(np.arange(0,20,1),
             train_f1, label="Train")
axes[1].plot(np.arange(0,20,1),
             val_f1,
             linestyle='--', color='g', alpha=.7,
             label="Validation")
axes[1].set_xticks(np.arange(0,20,5))
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("F1 score")
axes[1].set_title('Model F1 score through epochs',
                  color='#f48023', fontweight='bold')
axes[1].legend(loc=4)

plt.show()

In [ ]:
rnn_model.predict(X_test.toarray()).shape
#np.argmax(rnn_model.predict(X_test.toarray()), axis=-1)
#(rnn_model.predict(X_test.toarray()) > 0.5).astype("int32")